In [1]:
print('..running')
import os
import torch
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from util import cross_entropy_loss_fn
import models
import wandb
from train import evaluation, training 
from data import load_data

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

result_dir = 'models'
if not(os.path.exists(result_dir)):
    os.mkdir(result_dir)
name = 'made' #Change to regularized

batch_size = 64
D = 784   # input dimension
M = 8000  # hidden layer dimensionality
lr = 1e-3 # learning rate
num_epochs = 85
max_patience = 10 # Early Stopping
lam = 0.
n_masks = 1

hyperparameters = {'D': D, 
                   'M': M,
                   'lr': lr,
                   'n_masks': n_masks,
                   'num_epochs': num_epochs,
                   'max_patience': max_patience,
                   'batch_size': batch_size,
                   'lambda': lam,
                    }

run = wandb.init(entity="rajpal906")#entity="rajpal906", project="MADE", name="unregularized", id="1", config=hyperparameters, settings=wandb.Settings(start_method="fork"))
# Training procedure
train_data, val_data, test_data = load_data('mnist', binarize = True)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=os.cpu_count())
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=os.cpu_count())
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=os.cpu_count())
model = models.MADE(input_dim=D, hidden_dims=[M], n_masks=n_masks).to(device)
optimizer = torch.optim.Adam([p for p in model.parameters() if p.requires_grad == True], lr = lr)
scheduler = lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)
nll_val = training(name=name, result_dir=result_dir, max_patience=max_patience, num_epochs=num_epochs, 
                   model=model, loss_fn=cross_entropy_loss_fn, optimizer=optimizer, scheduler=scheduler, 
                   training_loader=train_loader, val_loader=val_loader, device=device, lam=lam, batch_size = batch_size)
model.eval()
test_val = evaluation(test_loader, cross_entropy_loss_fn, model_best=model)#_best)
wandb.log({"test_loss": test_val * batch_size})
run.log_artifact(result_dir + '/' + name + '.model')
run.finish()

..running


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: s2592586 (rajpal906). Use `wandb login --relogin` to force relogin


TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'